# Summary of Steps

## Replace Single Quotes with Double Quotes in the File
This was done using bash's sed command, replacing all __'__ with __"__ in order to conform to standard JSON. The original file with single quotes is saved as _records.log.bak_

## Read JSON String from File as Dictionaries
The file is loaded using the json library in python. It is then converted to a list of dictionaries, each of which represents an item in the log. 

## Convert to DataFrame
Then pandas' json_normalize() is applied to the list of dictionaries, normalizing each nested dictionary into a flat table. The output is a list of DataFrames, each of which contains a single order. To merge all the DataFrames into a single one, pandas' concat() is used.

## Perform Unit Conversion
In order to selectively convert all metric units to imperial units, the DataFrame method loc() is used to access the rows with metric measurements and convert them to imperial units. The imperial units indication column is then removed.

## Output to CSV
Finally, using pandas' to_csv(), the final form of the DataFrame is written to csv.

In [1]:
# Convert single quotes to double quotes (using bash's sed)
!sed -i.bak "s/'/\"/g" records.log

In [2]:
# import statements
import numpy as np
import pandas as pd
import json 

In [3]:
# reading the data from the file 
with open('records.log') as f: 
    data = f.readlines()   

In [4]:
# reconstructing the data as a list of dictionaries 
jsonString = json.dumps(data)

stringList = json.loads(jsonString)
dictList = [json.loads(string) for string in stringList]

In [5]:
#flatten dictionaries and convert into dataframes
flattenedDfList = [pd.json_normalize(dict) for dict in dictList]

In [6]:
# combine into dataframe
df = pd.concat(flattenedDfList)
df

,order_id,package.weight,package.volume,package.imperial_unit
0,21000,326,62018,false
0,21001,460,110592,false
0,21002,500,1440,false
0,21003,260,100700,true
0,21004,168,20700,true
...,...,...,...,...
0,21995,19,9882,false
0,21996,771,254404,false
0,21997,70,20160,false
0,21998,186,103125,false


In [7]:
# convert package.imperial_unit's dtype from object to string
df["package.imperial_unit"] = df["package.imperial_unit"].astype(str)

In [8]:
# convert all metric units in dataframe to imperial
df.loc[df["package.imperial_unit"] == "false", "package.weight"] =  df.loc[df["package.imperial_unit"] == "false", "package.weight"] * 2.204623

df.loc[df["package.imperial_unit"] == "false", "package.volume"] =  df.loc[df["package.imperial_unit"] == "false", "package.volume"] * 0.0610237


In [9]:
# remove imperial unit indication column
df.pop('package.imperial_unit')

0    false
0    false
0    false
0     true
0     true
     ...  
0    false
0    false
0    false
0    false
0    false
Name: package.imperial_unit, Length: 1000, dtype: object

In [10]:
# rename columns to match desired output
df.columns = ['order_id', 'weight (lbs)', 'volume (in3)']
df

,order_id,weight (lbs),volume (in3)
0,21000,718.707098,3784.567827
0,21001,1014.126580,6748.733030
0,21002,1102.311500,87.874128
0,21003,260.000000,100700.000000
0,21004,168.000000,20700.000000
...,...,...,...
0,21995,41.887837,603.036203
0,21996,1699.764333,15524.673375
0,21997,154.323610,1230.237792
0,21998,410.059878,6293.069062


In [11]:
# write to csv, removing the index
df.to_csv('records.csv', index = False)